# HW 8: example

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
mal = read_tsv("rawdata/allhaps.malathion.200kb.txt.gz")

Rows: 208128 Columns: 5
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): chr, pool, founder
dbl (2): pos, freq

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
# select the first position in the genome
mal2 = mal %>% filter(chr=="chrX" & pos==316075)
mal2
levels(as.factor(mal2$pool))
levels(as.factor(mal2$founder))

chr,pos,pool,founder,freq
<chr>,<dbl>,<chr>,<chr>,<dbl>
chrX,316075,mcF,A1,0.0077
chrX,316075,mcF,A2,0.0003
chrX,316075,mcF,A3,0.0527
chrX,316075,mcF,A4,0.6318
chrX,316075,mcF,A5,0.0804
chrX,316075,mcF,A6,0.0338
chrX,316075,mcF,A7,0.1700
chrX,316075,mcF,AB8,0.0232
chrX,316075,mcM,A1,0.0064


[1] "mcF" "mcM" "msF" "msM"

[1] "A1"  "A2"  "A3"  "A4"  "A5"  "A6"  "A7"  "AB8"

In [4]:
mal2 = mal2 %>% mutate(treat=str_sub(pool,2,2))
anova(lm(asin(sqrt(freq)) ~ treat + founder + treat:founder, data=mal2))

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
treat,1,0.0006243635,0.0006243635,0.2760955,6.064778e-01
founder,7,2.0425777430,0.2917968204,129.0334579,7.195360e-13
treat:founder,7,0.0348952369,0.0049850338,2.2043974,9.027393e-02
Residuals,16,0.0361824693,0.0022614043,NA,NA


# Problem 1

In [1]:
# Load necessary library
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Read the data
mal <- read_tsv("rawdata/allhaps.malathion.200kb.txt.gz")

Rows: 208128 Columns: 5
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): chr, pool, founder
dbl (2): pos, freq

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
# Prepare the data by extracting treatment information
mal = mal %>% mutate(treat = str_sub(pool, 2, 2))

# Group by chromosome and position, then nest
nested_mal = mal %>%
  group_by(chr, pos) %>%
  nest()

In [4]:
# Function to fit the model and extract the p-value
fit_model_extract_p <- function(data) {
  # Check if there are at least two levels for each factor
  if (nlevels(factor(data$treat)) < 2 || nlevels(factor(data$founder)) < 2) {
    return(NA)  # Return NA if not enough levels
  }
  
  # Try fitting the model and extracting the p-value
  tryCatch({
    model <- lm(asin(sqrt(freq)) ~ treat + founder + treat:founder, data = data)
    anova_result <- anova(model)
    p_value <- anova_result["treat:founder", "Pr(>F)"]
    return(p_value)
  }, error = function(e) {
    return(NA)  # Return NA if there's an error
  })
}

In [5]:
# Apply the function to each nested data frame and calculate -log10(p)
results <- nested_mal %>%
  mutate(p_value = map_dbl(data, fit_model_extract_p)) %>%
  mutate(log10_p = -log10(p_value))

# View the results
results %>% select(chr, pos, log10_p)

chr,pos,log10_p
<chr>,<dbl>,<dbl>
chrX,316075,1.0444377
chrX,336075,0.9820351
chrX,356075,1.1715317
chrX,376075,1.0378760
chrX,396075,0.8694061
chrX,416075,0.7932918
chrX,436075,0.8200451
chrX,456075,0.7239865
chrX,476075,0.6433032


In [6]:
# Select relevant columns
output_results <- results %>% select(chr, pos, log10_p)

# Save the results to a CSV file
write_csv(output_results, "genome_analysis_results.csv")

# Optionally, print a message indicating the file was saved
cat("Results have been saved to 'genome_analysis_results.csv'\n")

Results have been saved to 'genome_analysis_results.csv'


# Problem 2

In [1]:
# Load necessary libraries
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Read the dataset
mal = read_tsv("rawdata/allhaps.malathion.200kb.txt.gz")

Rows: 208128 Columns: 5
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): chr, pool, founder
dbl (2): pos, freq

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
# Prepare the data by extracting treatment information
mal = mal %>% mutate(treat = str_sub(pool, 2, 2))

In [4]:
# Group by chromosome and position, then nest
nested_mal = mal %>%
  group_by(chr, pos) %>%
  nest()

In [5]:
# Function to fit the new model and extract the p-value
fit_new_model_extract_p <- function(data) {
  # Check if there are at least two levels for each factor
  if (nlevels(factor(data$treat)) < 2 || nlevels(factor(data$founder)) < 2) {
    return(NA)  # Return NA if not enough levels
  }
  
  # Try fitting the new model and extracting the p-value
  tryCatch({
    model <- lm(asin(sqrt(freq)) ~ founder + treat %in% founder, data = data)
    anova_result <- anova(model)
    p_value <- anova_result["treat:founder", "Pr(>F)"]
    return(p_value)
  }, error = function(e) {
    return(NA)  # Return NA if there's an error
  })
}

In [6]:
# Apply the function to each nested data frame and calculate -log10(p)
results_new_model <- nested_mal %>%
  mutate(p_value = map_dbl(data, fit_new_model_extract_p)) %>%
  mutate(log10_p = -log10(p_value))

In [7]:
# Select relevant columns
output_results_new_model <- results_new_model %>% select(chr, pos, log10_p)

In [9]:
# View the results
results_new_model %>% select(chr, pos, log10_p)

chr,pos,log10_p
<chr>,<dbl>,<dbl>
chrX,316075,NA
chrX,336075,NA
chrX,356075,NA
chrX,376075,NA
chrX,396075,NA
chrX,416075,NA
chrX,436075,NA
chrX,456075,NA
chrX,476075,NA


In [8]:
# Save the results to a CSV file
write_csv(output_results_new_model, "genome_analysis_results_new_model.csv")

# Optionally, print a message indicating the file was saved
cat("Results have been saved to 'genome_analysis_results_new_model.csv'\n")

Results have been saved to 'genome_analysis_results_new_model.csv'


# Problem 3

In [1]:
# Load necessary libraries
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Read the results from the two CSV files
results_first_model <- read_csv("genome_analysis_results.csv")
results_second_model <- read_csv("genome_analysis_results_new_model.csv")

Rows: 6504 Columns: 3
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (1): chr
dbl (2): pos, log10_p

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 6504 Columns: 3
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (1): chr
dbl (1): pos
lgl (1): log10_p

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
# Rename the log10_p columns to distinguish between the two models
results_first_model <- results_first_model %>% rename(log10_p_first_model = log10_p)
results_second_model <- results_second_model %>% rename(log10_p_second_model = log10_p)

In [4]:
# Merge the two results using an inner join on chr and pos
merged_results <- inner_join(results_first_model, results_second_model, by = c("chr", "pos"))

# View the merged results
print(merged_results)

# A tibble: 6,504 × 4
   chr      pos log10_p_first_model log10_p_second_model
   <chr>  <dbl>               <dbl> <lgl>               
 1 chrX  316075               1.04  NA                  
 2 chrX  336075               0.982 NA                  
 3 chrX  356075               1.17  NA                  
 4 chrX  376075               1.04  NA                  
 5 chrX  396075               0.869 NA                  
 6 chrX  416075               0.793 NA                  
 7 chrX  436075               0.820 NA                  
 8 chrX  456075               0.724 NA                  
 9 chrX  476075               0.643 NA                  
10 chrX  496075               0.575 NA                  
# ℹ 6,494 more rows


In [5]:
# Save the merged results to a new CSV file
write_csv(merged_results, "merged_genome_analysis_results.csv")

# Print a message indicating the file was saved
cat("Merged results have been saved to 'merged_genome_analysis_results.csv'\n")

Merged results have been saved to 'merged_genome_analysis_results.csv'
